# Uncalibrated Model Outputs

In [ ]:
model_path = None
observations_path = None
timeout = 240
parameters_path = '../data/parameters/testbed.csv'

In [ ]:
import pandas as pd

from mhpc_project.utils import date_parser
from mhpc_project.plots import comparisons
from mhpc_project.parameters import UniformSoilParameters as Parameters
from mhpc_project.models import UniformSoilModel as Model

In [ ]:
parameters = Parameters(parameters_path, {})
model = Model(model_path, timeout=timeout)
observations = pd.read_csv(observations_path,
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0)
predictions = model(parameters.init)

In [ ]:
comparisons(model, observations)
